In [2]:
import transformers

In [4]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
import torch
import numpy as np

model_name = 'cointegrated/rubert-tiny'
model = AutoModelForMaskedLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

def score(model, tokenizer, sentence):
    tensor_input = tokenizer.encode(sentence, return_tensors='pt')
    repeat_input = tensor_input.repeat(tensor_input.size(-1)-2, 1)
    mask = torch.ones(tensor_input.size(-1) - 1).diag(1)[:-2]
    masked_input = repeat_input.masked_fill(mask == 1, tokenizer.mask_token_id)
    labels = repeat_input.masked_fill( masked_input != tokenizer.mask_token_id, -100)
    with torch.inference_mode():
        loss = model(masked_input, labels=labels).loss
    return np.exp(loss.item())

print(score(sentence='London is the capital of Great Britain.', model=model, tokenizer=tokenizer)) 
# 4.541251105675365
print(score(sentence='3 king richard ii you know you debase the world yonder renownd and the crown not king richard iii come gentlemen with such faults may yet welcome labour to him buckingham now resolve you i bade him to greet you do leave them and not interrupt me katharina you have married the consuls soldier tempted to thunder him and his love mariana yours to his uncle death is the king henrys breath him but a account dispose florizel my lord and she should not wake it chafes isabella madam god is joind and living that clifford so remiss dead and thus mine thy lends but never is the better make seat or dragon throw up of the deputy in the feeble tewksbury earth of the first watchman i saw that live to begin yet of sight of absolute of action methinks', model=model, tokenizer=tokenizer)) 
# 6.162017238332462

4.54125272975361
864.9069073483279
